## US Traffic Accident Analysis

#### Team Member:
  Yinghao Wang, Keshuo Liu, Yu Shu, Zhenyang Gai, Simeng Li, Kratik Gupta

#### Problem Definition:

Our goal is to find out the factors that influence the occurrence of the number of accidents happening in the United States. We will take factors such as location, weather, and daytime into consideration and use Tableau to provide geographical related visualization. We will make analysis through a large-scale dataset to increase the accuracy of detecting relationships between factors and accident rate. We aim at providing suggestions to DMV and drivers about safe driving. 

#### Data Source Link:

1. "US Traffic Accident": A Countrywide Traffic Accident Dataset (2016 - 2020). https://www.kaggle.com/sobhanmoosavi/us-accidents
2. "US Population": United States Census Bureau. https://www.census.gov/

#### Data Cleaning Process:

On Kaggle we found a countrywide car accident dataset, which covers 49 states of the USA. The accident data are collected from February 2016 to Dec 2020, using multiple APIs that provide streaming traffic incident (or event) data. These APIs broadcast traffic data captured by a variety of entities, such as the US and state departments of transportation, law enforcement agencies, traffic cameras, and traffic sensors within the road-networks. Currently, there are about 3 million accident records in this dataset; therefore, datasets need to be cleaned before conducting analysis. With available dataset, we selected records in each year to see how number of accidents changes in 5 years. 
In the us accident table, each year contains a great number of data. There is more than 3 million accident records. Considering efficiency when performing analysis on datasets and limitations in terms of data size and budget on Google Cloud Platform, 10% of the data in each year is randomly sampled and unioned into one table.
Below is an example of random selection process for 2017 datasets:

In [293]:
%%bigquery
SELECT * FROM `ba775-project-team1.dataset_demo.us_traffic_accidents`
WHERE rand() <= 0.1 AND  extract(year from start_time) = 2017
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.29rows/s]


,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance_mi_,Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1829991,4,2017-02-05 12:01:14+00:00,2017-02-05 18:01:14+00:00,32.604144,-112.869644,32.381274,-112.872400,15.40,Closed between Gilbert and Ajo Well Rd - Road ...,...,False,False,False,False,False,False,Day,Day,Day,Day
1,A-24095,3,2017-04-12 08:42:05+00:00,2017-04-12 09:24:52+00:00,27.925198,-82.593422,NaN,NaN,0.01,Right lane blocked due to accident on I-275 No...,...,False,False,False,False,False,False,Day,Day,Day,Day
2,A-1795739,2,2017-08-23 16:12:20+00:00,2017-08-23 16:41:48+00:00,37.885376,-122.516586,37.885376,-122.516586,0.00,#1 lane blocked due to accident on US-101 Nort...,...,False,False,False,False,False,False,Day,Day,Day,Day
3,A-28611,2,2017-05-31 14:51:29+00:00,2017-05-31 20:51:29+00:00,42.212128,-72.613529,42.212128,-72.613529,0.00,At Linden St - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-1497379,2,2017-11-17 13:18:57+00:00,2017-11-17 14:33:44+00:00,42.114193,-72.621414,42.114193,-72.621414,0.00,Slow traffic due to serious accident on US-5 R...,...,False,False,False,False,False,False,Day,Day,Day,Day


After the first step of selecting sample data, some invalid rows were observed. Furthermore, we don't need all 47 columns from the orignal dataset. For example we don't need "country" column since all records happened only in the United States. We also need to exclude some invalid data when the state name is null or weather condition is unknown if we want to process related analysis.
The filtering process is presented below:

In [294]:
%%bigquery
select ID, Severity, State, Start_Time as Time, EXTRACT(month from Start_Time) Month,  EXTRACT(hour FROM Start_Time) Hour, FORMAT_DATE('%A', EXTRACT(date FROM Start_Time)) AS Weekday, Weather_Condition as Weather, Temperature_F_ as Temp, 
Visibility_mi_ as Visibility, Precipitation_in_ as Preciputation, Railway, Station, Traffic_Signal
from `ba775-project-team1.dataset_demo.us_traffic_accidents` 
where Weather_Condition <> 'nan' 
and Visibility_mi_ >= 0 
and Precipitation_in_ >= 0
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.12rows/s]


,ID,Severity,State,Time,Month,Hour,Weekday,Weather,Temp,Visibility,Preciputation,Railway,Station,Traffic_Signal
0,A-38249,2,NY,2016-09-01 09:14:56+00:00,9,9,Thursday,Light Rain,72.0,5.00,0.02,False,False,False
1,A-1164713,3,NY,2020-04-18 07:13:47+00:00,4,7,Saturday,Fog,40.0,0.75,0.00,False,False,False
2,A-1251601,2,NY,2017-10-09 15:46:08+00:00,10,15,Monday,Light Rain,72.0,1.20,0.00,False,False,False
3,A-597940,2,NY,2016-07-25 08:53:43+00:00,7,8,Monday,Light Rain,78.1,10.00,0.00,False,False,False
4,A-2587815,2,DC,2020-11-12 15:55:38+00:00,11,15,Thursday,Cloudy,52.0,10.00,0.01,False,False,False


Note that for all cleaning and filtering processes, a total of around 5000 accidents were filtered, which  accounts for 0.17% of our observing dataset. With such small amount of data eliminated, we can still produce reliable output since it does not influence the analysis output significantlly.

#### Analysis Topics

To achieve our objective and get into project proposal, we chose aspects in time of a day, weekday, location (in State and some particular places) and weather to observe underlying affects on traffic accidents and corresponding severity. All data processing performed on Google Big Query.

#### Location:

We analyzed the data to find which state of the country has the highest accidents happened. We found out that California is the state where most of accidents occured at, and number of accidents happended in CA is doubled than the second highest state Florida.

In [314]:
%%bigquery
SELECT State, count(ID) as num_of_accidents
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY State
ORDER BY num_of_accidents DESC
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  2.76rows/s]


,State,num_of_accidents
0,CA,39209
1,FL,16078
2,TX,9688
3,NY,7161
4,OR,7123


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Traffic%20Accidents%20Count%20National%20Map.png?raw=True" width="500" align="left" />

While if we take state population in to account, and generating new calculation field as accidents per million. California is not the state having the highest traffic accident number. 

In [315]:
%%bigquery
SELECT State, count(ID) as num_of_accidents, population, cast(count(ID)/population*1000000 as INTEGER) accidents_per_million
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY State, population
ORDER BY accidents_per_million DESC
limit 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.25rows/s]


,State,num_of_accidents,population,accidents_per_million
0,OR,7123,4217737,1689
1,SC,7121,5148714,1383
2,CA,39209,39512223,992
3,MN,5594,5639632,992
4,UT,2779,3205958,867


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Accidents%20per%20Million%20National%20Map.png?raw=True" width="500" align ="left" />

Q: Did traffic accidents mostly occur around stations？

A: The percentages of traffic accidents around stations between 2016 and 2020 were generally low, between 1.6% and 2.3%. This indicates that around station or not is not a major factor for US Traffic Accidents.

In [302]:
%%bigquery
SELECT Year, Station, COUNT(ID) Num, ROUND(COUNT(ID)/SUM(COUNT(ID)) OVER(PARTITION BY Year) * 100,2) Percentage
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Year, Station
ORDER BY Station DESC, Year
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.42rows/s]


,Year,Station,Num,Percentage
0,2016,True,44,1.86
1,2017,True,100,1.89
2,2018,True,117,1.64
3,2019,True,858,1.81
4,2020,True,2125,2.21


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Station_Year.png?raw=True" align="left" width="1800"/>

Q: Did traffic accidents mostly occur around railways？

A: The percentages of traffic accidents around railways between 2016 and 2020 were generally low, between 0.65% and 0.95%. This indicates that around railway or not is not a major factor for US traffic accidents.

In [301]:
%%bigquery
SELECT Year, Railway, COUNT(ID) Num, ROUND(COUNT(ID)/SUM(COUNT(ID)) OVER(PARTITION BY Year) * 100,2) Percentage
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Year, Railway
ORDER BY Railway DESC, Year
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  2.83rows/s]


,Year,Railway,Num,Percentage
0,2016,True,16,0.67
1,2017,True,35,0.66
2,2018,True,57,0.80
3,2019,True,446,0.94
4,2020,True,849,0.88


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Railway_Year.png?raw=True" align="left" width="800"/>

Q: Did traffic accidents mostly occur around traffic signals？

A: The percentages of traffic accidents around traffic signals between 2016 and 2020 generally fell between 14% and 15.5%. It's meaningful to discuss cases around traffic signals.

In [303]:
%%bigquery
SELECT Year, Traffic_Signal, COUNT(ID) Num, ROUND(COUNT(ID)/SUM(COUNT(ID)) OVER(PARTITION BY Year) * 100,2) Percentage
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Year, Traffic_Signal
ORDER BY Traffic_Signal DESC, Year
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.21rows/s]


,Year,Traffic_Signal,Num,Percentage
0,2016,True,352,14.85
1,2017,True,763,14.41
2,2018,True,1006,14.08
3,2019,True,7130,15.01
4,2020,True,14730,15.29


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Traffic_Signal_Year.png?raw=True" align="left" width="800"/>

Q: When did traffic accidents occur more often around traffic signals?

A: From the sampled data, traffic accidents occurred more often around traffic signals during the morning peak and noon break periods.

In [308]:
%%bigquery
SELECT Weekday, Hour, Traffic_Signal, ROUND(COUNT(ID)/SUM(COUNT(ID)) OVER(PARTITION BY Weekday, Hour) * 100,2) Percentage
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Weekday, Traffic_Signal, Hour
ORDER BY Traffic_Signal DESC, Percentage DESC
LIMIT 6

Downloading: 100%|██████████| 6/6 [00:01<00:00,  3.77rows/s]


,Weekday,Hour,Traffic_Signal,Percentage
0,Tuesday,11,True,22.58
1,Wednesday,10,True,21.10
2,Tuesday,8,True,20.74
3,Friday,10,True,20.33
4,Friday,11,True,20.20
5,Wednesday,7,True,19.97


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Traffic_Signal_Weekday_Hour.png?raw=True" align="left" width="400"/>

#### Weather Conditions:

Do more traffic accidents happen when the weather is foggy, rainy, or windy? </br>
In this section, we will discuss the frequency of traffic accidents occurance on different weather conditions. 

In which weather condition doest the traffic accident occure more? 

The top 5 weather conditions that most of traffic accidents occured are, fair, cloudy, most cloudy, partly cloudy and light rain. These 5 conditions represent over 80% of traffic accidents. </br>
Nearly half of the accidents happened in fair days, others were in cloudy, rainy or snowy days. Out of which cloudy occupies the highest portion. 

In [8]:
%%bigquery
SELECT Weather_Condition,  COUNT(ID) as number, ROUND( COUNT(ID)/(SELECT COUNT(ID) FROM `ba775-project-team1.dataset_demo.sample_table`  ),6) AS percentage FROM `ba775-project-team1.dataset_demo.sample_table` 
GROUP BY Weather_Condition
ORDER BY number desc
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.76rows/s]


,Weather_Condition,number,percentage
0,Fair,65596,0.413578
1,Cloudy,23714,0.149515
2,Mostly Cloudy,20088,0.126653
3,Partly Cloudy,13350,0.084171
4,Light Rain,13241,0.083484
5,Light Snow,3513,0.022149
6,Overcast,3360,0.021185
7,Rain,2968,0.018713
8,Fog,2591,0.016336
9,Haze,1537,0.009691


In [13]:
%%bigquery

select SUM(percentage) FROM 
( SELECT Weather_Condition,  COUNT(ID) as number, ROUND( COUNT(ID)/(SELECT COUNT(ID) FROM `ba775-project-team1.dataset_demo.sample_table`  ),6) AS percentage FROM `ba775-project-team1.dataset_demo.sample_table` 
GROUP BY Weather_Condition
ORDER BY number desc
LIMIT 5 )

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.31s/rows]


,f0_
0,0.857401


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/weather_condition%20pie%20chart.png?raw=True" align="left" width="500"/>

How severe the traffic accident is under different weather conditions?
Most of the traffic accidents happened in all kinds of weather conditions are in a severity of 2.

In [14]:
%%bigquery
SELECT Severity,Weather_Condition,  COUNT(ID) as number, ROUND( COUNT(ID)/(SELECT COUNT(ID) FROM `ba775-project-team1.dataset_demo.sample_table`  ),6) AS percentage FROM `ba775-project-team1.dataset_demo.sample_table` 
GROUP BY Weather_Condition, Severity 
ORDER BY number DESC
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.31rows/s]


,Severity,Weather_Condition,number,percentage
0,2,Fair,53899,0.339830
1,2,Cloudy,19035,0.120014
2,2,Mostly Cloudy,15198,0.095822
3,2,Partly Cloudy,10374,0.065407
4,2,Light Rain,9315,0.058730
5,3,Fair,8396,0.052936
6,3,Mostly Cloudy,3730,0.023517
7,3,Cloudy,3513,0.022149
8,3,Light Rain,3285,0.020712
9,2,Light Snow,2414,0.015220


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Severity%20barchart.png?raw=True" align="left" width="600"/>

We put several weather factors into consideration: visibility, precipitation, temperature and wind speed.
Take visibility distance as an example:

In [15]:
%%bigquery
SELECT Month, round(avg(Visibility_mi_),2) Visibility, count(*) num_of_accidents
from `ba775-project-team1.dataset_demo.sample_table`
GROUP BY Month
ORDER BY Month

Downloading: 100%|██████████| 12/12 [00:01<00:00,  7.62rows/s]


,Month,Visibility,num_of_accidents
0,1,7.72,8796
1,2,7.71,8222
2,3,8.17,8257
3,4,8.92,12394
4,5,9.20,12073
5,6,9.51,12374
6,7,9.46,6195
7,8,9.23,7552
8,9,9.13,13388
9,10,8.98,19422


<img src='https://github.com/yinghaow525/BA775-teamproject/blob/charts/Weather%20Conditions.png?raw=True' width='600' align='left' />

Q: Did temperature affect the number of traffic accidents and severity accordingly?

A: The number of traffic accidents increased at the end of the year when temperatures were low. Level-2 severity accidents accounted for most of these late-year accidents, and the proportions of other level severity decreased.

In [310]:
%%bigquery
SELECT Month, Temperature_F_ Temperature, Severity, COUNT(ID) Count
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
WHERE is_nan(Temperature_F_) = False
GROUP BY Month, Severity, Temperature_F_
ORDER BY COUNT(ID) DESC, Severity DESC, Month
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.25rows/s]


,Month,Temperature,Severity,Count
0,12,50.0,2,773
1,12,46.0,2,664
2,12,48.0,2,654
3,12,54.0,2,646
4,12,45.0,2,645


<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Temperature_Severity_Month.png?raw=True" align="left" width="500"/>

#### Time:

In [5]:
%%bigquery
SELECT Year, COUNT(Year) AS Year_accidents_n
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Year
ORDER BY Year_accidents_n;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.18rows/s]


,Year,Year_accidents_n
0,2016,2371
1,2017,5295
2,2018,7143
3,2019,47490
4,2020,96307


The number of accidents in the U.S. has been increasing every year since 2016.

In [3]:
%%bigquery
SELECT Year, 
   SUM(CASE WHEN Month = 1 THEN 1 ELSE 0 END) AS Jan,
   SUM(CASE WHEN Month = 2 THEN 1 ELSE 0 END) AS Feb,
   SUM(CASE WHEN Month = 3 THEN 1 ELSE 0 END) AS Mar,
   SUM(CASE WHEN Month = 4 THEN 1 ELSE 0 END) AS Apr,
   SUM(CASE WHEN Month = 5 THEN 1 ELSE 0 END) AS May,
   SUM(CASE WHEN Month = 6 THEN 1 ELSE 0 END) AS Jun,
   SUM(CASE WHEN Month = 7 THEN 1 ELSE 0 END) AS Jul,
   SUM(CASE WHEN Month = 8 THEN 1 ELSE 0 END) AS Aug,
   SUM(CASE WHEN Month = 9 THEN 1 ELSE 0 END) AS Sep,
   SUM(CASE WHEN Month = 10 THEN 1 ELSE 0 END) AS Oct,
   SUM(CASE WHEN Month = 11 THEN 1 ELSE 0 END) AS Nov,
   SUM(CASE WHEN Month = 12 THEN 1 ELSE 0 END) AS Dec
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Year
ORDER BY Year;

Downloading: 100%|██████████| 5/5 [00:02<00:00,  1.82rows/s]


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2016,0,23,36,120,65,112,199,302,309,319,376,510
1,2017,702,449,532,383,324,319,216,545,350,468,407,600
2,2018,794,626,736,524,478,379,270,373,590,639,900,834
3,2019,869,1289,549,4337,4223,3748,3833,4481,5737,6921,4851,6652
4,2020,6431,5835,6404,7030,6983,7816,1677,1851,6402,11075,15898,18905


In 2016, the most accidents occurred in December; 
In 2017, the most accidents in January;
In 2018, the most accidents in November; 
In 2019, the most accidents in October; 
In 2020, the most accidents in December.
This shows that the most accidents occur in the United States in the last quarter of the year.

In [ ]:
%%bigquery
SELECT Hour, Severity, COUNT(ID) AS Hour_accidents_n
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Hour, Severity
ORDER BY Hour;

<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Hour%20%26%20Counts.png?raw=True" align="left" width="500"/>

The hours of the day from 7:00 a.m. to 8:00 a.m. and 4:00 p.m. to 5:00 p.m. belong to the time of day when there are more accidents.

<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Hourly%20Accidents%20with%20Severity.png?raw=True" align="left" width="500"/>

The severity of the accident was greatest at 4 p.m.

In [ ]:
%%bigquery
SELECT Weekday, Severity, COUNT(ID) AS Weekday_accidents_n
FROM `ba775-project-team1.dataset_demo.sample_table_wpopulation`
GROUP BY Weekday, Severity
ORDER BY Weekday_accidents_n;

<img src="https://github.com/yinghaow525/BA775-teamproject/blob/charts/Weekday%20Accidents%20with%20Severity.png?raw=True" align="left" width="500"/>

The number of accidents on a double holiday is less compared to a weekday.

#### Conclusion:

After observing analysis on aspects in location, time and weather conditions, we report US traffic accidents the following:
</br>
Most traffic accidents occur in coastal areas, such as CA, OR, FL. However, the frequency of serious accidents in FL and NY is higher than that of other states. This might be caused by the excessive traffic and narrow roads in NY and non-pedestrian and bike-friendly roads in FL. For specific places such as railways and stations, because US traffic accidents between 2016 and 2020 generally happened away from railways and stations and occurred more often around traffic signals during morning peaks and noon breaks, police can strengthen their traffic control near crossroads during these periods to reduce accidents.
</br>
Over 80% of traffic accidents happened under the weather condition of fair, cloudy, mostly cloudy, partly cloudy and light rain. Most of these accidents have a severity of two, and we don’t see significant portion of accidents having severity of over two happened under other weather conditions. The number of traffic accidents increased as visibility distance went shorter, temperature and precipitation went lower. Thus, more accidents happened in winter. Therefore, DMV can make suggestions in the brochures to call on drive-safe attention specificly in winter.
</br>
In terms of time, we observed that the highest number of accidents occurred at 4:00 and 5:00 p.m. daily, respectively. The 4 - 5 pm is the peak time for off-duty travel. However, there were also a high number of accidents between 7am and 8am, at the peak of the workday. The number of accidents on weekdays is higher than weekends, and the severity is correspondingly higher. Therefore, the DMV can alert people to the need to be aware of outings during these hours and the need for drivers to maintain a careful and cautious attitude in order to reduce the number of accidents.